In [1]:
import numpy as np
import pandas as pd

In [66]:
def gaussian_ellipsoid_props(cov, sdwidth=None):
    """
    get the scaling, rotation of the ellipsoid
    """
    eigvals, eigvecs = np.linalg.eig(cov)
    scaling = sdwidth * np.sqrt(eigvals)
    rotation = roll_pitch_yaw(eigvecs)
    return scaling, rotation

In [67]:
def roll_pitch_yaw(R):
    """
    Illustration of the rotation matrix / sometimes called 'orientation' matrix
    R = [
           R11 , R12 , R13,
           R21 , R22 , R23,
           R31 , R32 , R33
        ]
    REMARKS:
    1. this implementation is meant to make the mathematics easy to be deciphered
    from the script, not so much on 'optimized' code.
    You can then optimize it to your own style.
    2. I have utilized naval rigid body terminology here whereby;
    2.1 roll -> rotation about x-axis
    2.2 pitch -> rotation about the y-axis
    2.3 yaw -> rotation about the z-axis (this is pointing 'upwards')
    """
    from math import (
        asin, pi, atan2, cos
    )
    R11 = R[0,0]
    R12 = R[0,1]
    R13 = R[0,2]

    R21 = R[1,0]
    R22 = R[1,1]
    R23 = R[1,2]

    R31 = R[2, 0]
    R32 = R[2, 1]
    R33 = R[2, 2]


    if R31 != 1 and R31 != -1:
        pitch_1 = -1 * asin(R31)
        pitch_2 = pi - pitch_1
        roll_1 = atan2(R32 / cos(pitch_1), R33 / cos(pitch_1))
        roll_2 = atan2(R32 / cos(pitch_2), R33 / cos(pitch_2))
        yaw_1 = atan2(R21 / cos(pitch_1), R11 / cos(pitch_1))
        yaw_2 = atan2(R21 / cos(pitch_2), R11 / cos(pitch_2))

        # IMPORTANT NOTE here, there is more than one solution but we choose the first for this case for simplicity !
        # You can insert your own domain logic here on how to handle both solutions appropriately (see the reference publication link for more info).
        pitch = pitch_1
        roll = roll_1
        yaw = yaw_1
    else:
        yaw = 0  # anything (we default this to zero)
        if R31 == -1:
            pitch = pi / 2
            roll = yaw + atan2(R12, R13)
        else:
            pitch = -pi / 2
            roll = -1 * yaw + atan2(-1 * R12, -1 * R13)

    # convert from radians to degrees
    roll = roll * 180 / pi
    pitch = pitch * 180 / pi
    yaw = yaw * 180 / pi

    return [roll, pitch, yaw]

In [68]:
cov = np.eye(3) * np.array([1,2,3])
cov

array([[1., 0., 0.],
       [0., 2., 0.],
       [0., 0., 3.]])

In [69]:
gaussian_ellipsoid_props(cov, sdwidth=3)

(array([3.        , 4.24264069, 5.19615242]), [0.0, -0.0, 0.0])

In [70]:
np.sqrt(3) * 3

5.196152422706632

In [71]:
# Sample from:
d = 3 # Number of dimensions
mean = np.matrix([[0.], [1.], [2.]])
covariance = np.matrix([
    [ 1.0, 0.8,  0.5], 
    [ 0.8, 1.0,  0.4],
    [ 0.5, 0.4,  1.0]
])

# Create L
L = np.linalg.cholesky(covariance)
# Sample X from standard normal
n = 5000  # Samples to draw
X = np.random.normal(size=(d, n))
# Apply the transformation
Y = L.dot(X) + mean


In [72]:
Y.T

matrix([[-0.21151026,  1.37782697,  2.21241187],
        [-1.37984787, -0.05746648,  1.06104902],
        [ 0.23122818,  1.17039376,  2.31647355],
        ...,
        [ 0.56229915,  2.11807091,  1.86661835],
        [ 0.25624951,  0.87167439,  3.61280318],
        [ 1.79771875,  1.63699606,  1.64952801]])

In [73]:
cov_hat = np.cov(Y)
cov_hat

array([[1.03245147, 0.83361098, 0.50172152],
       [0.83361098, 1.02847484, 0.39492964],
       [0.50172152, 0.39492964, 0.98213103]])

In [103]:
rho_xy = 0.0
sigma_x = 2
sigma_y = 3
sigma_z = 4
my_cov = [
    [sigma_x**2, rho_xy*sigma_x*sigma_y, 0],
    [rho_xy*sigma_x*sigma_y, sigma_y**2, 0],
    [0, 0, sigma_z]
    
]
scaling, rotation = gaussian_ellipsoid_props(3.407345007480164 * np.array(my_cov), sdwidth=1)

In [104]:
scaling

array([3.69179902, 5.53769854, 3.69179902])

In [44]:
rotation

[8.098272227944676, 28.23296714865301, -135.731971156704]

3.22172324

In [78]:
scaling[0]/2

2.4162924315167267

In [79]:
scaling[1]/3

3.225697573851791

1.4142135623730951

In [86]:
scaling**2

array([ 2.59487516, 10.40512484,  4.        ])

In [89]:
p = 0.99
2*np.log(1−p)

SyntaxError: invalid character in identifier (<ipython-input-89-dfeba8ef3d14>, line 2)

In [96]:
p = 0.997
-2 * np.log(1-p)

11.618285980628054

In [97]:
np.sqrt(11.61)

3.407345007480164

In [ ]:
n